In [1]:
# Importing relevant librairies
import numpy as np
import pandas as pd
import yfinance as yf
import os
pd.set_option('display.max_columns', None)

# Importing project librairies
from dynamic_portfolio.utils import load_csv, return_tickers

In [2]:
test = load_csv('V')

/home/racho961/code/dynamic_portfolio/dynamic_portfolio/utils.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['return'][0]=0


In [5]:
test.head()

,date,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,reportedEPS,surprisePercentage,10Y_yield,2Y_yield,10_2_spread,oil_price,orders,fed_funds,unemployment_rate,inf_exp,non_farm_payroll,CPI,retail_sales,gdp_per_capita,gold_price,usd_price,return
0,2008-04-28,76.22,77.29,74.90,75.63,17.040247,33129300.0,0.0,1.0,0.13,18.1818,0.0386,0.0236,0.0150,118.78,225075.0,0.0229,5.0,0.048,137896.0,214.823,331002.0,51923.0,892.50,72.790001,0.000000
1,2008-04-29,73.28,80.88,71.00,80.88,18.223128,49327100.0,0.0,1.0,0.13,0.0000,0.0385,0.0235,0.0150,115.67,225075.0,0.0221,5.0,0.048,137896.0,214.823,331002.0,51923.0,870.05,72.889999,0.069417
2,2008-04-30,81.21,84.81,81.07,83.45,18.802177,36885900.0,0.0,1.0,0.13,0.0000,0.0377,0.0229,0.0148,113.70,225075.0,0.0237,5.0,0.048,137896.0,214.823,331002.0,51923.0,869.95,72.510002,0.031775
3,2008-05-01,84.68,87.64,83.90,85.40,19.241532,29098900.0,0.0,1.0,0.13,0.0000,0.0378,0.0237,0.0141,112.60,227995.0,0.0216,5.4,0.052,138470.0,216.632,357277.0,51923.0,851.65,73.279999,0.023367
4,2008-05-02,87.25,87.50,81.00,82.75,18.644459,34162200.0,0.0,1.0,0.13,0.0000,0.0389,0.0247,0.0142,116.36,227995.0,0.0188,5.4,0.052,138470.0,216.632,357277.0,51923.0,855.45,73.500000,-0.031030


In [4]:
test.dtypes

date                   object
open                  float64
high                  float64
low                   float64
close                 float64
adjusted_close        float64
volume                float64
dividend_amount       float64
split_coefficient     float64
reportedEPS           float64
surprisePercentage    float64
10Y_yield             float64
2Y_yield              float64
10_2_spread           float64
oil_price             float64
orders                float64
fed_funds             float64
unemployment_rate     float64
inf_exp               float64
non_farm_payroll      float64
CPI                   float64
retail_sales          float64
gdp_per_capita        float64
gold_price            float64
usd_price             float64
return                float64
dtype: object

In [7]:
test.describe()

,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,reportedEPS,surprisePercentage,10Y_yield,2Y_yield,10_2_spread,oil_price,orders,fed_funds,unemployment_rate,inf_exp,non_farm_payroll,CPI,retail_sales,gdp_per_capita,gold_price,usd_price,return
count,3634.000000,3634.000000,3634.000000,3634.000000,3634.000000,3.634000e+03,3634.000000,3634.000000,3634.000000,3634.000000,3634.000000,3634.000000,3634.000000,3634.000000,3634.000000,3634.000000,3634.000000,3634.000000,3634.000000,3634.000000,3634.000000,3634.000000,3634.000000,3634.000000,3634.000000
mean,136.415916,137.759429,135.065245,136.436566,89.629952,7.143688e+06,0.003941,1.000826,0.772647,0.101987,0.023818,0.010360,0.013457,71.919417,220822.482939,0.006155,6.347606,0.031501,140845.070171,241.780550,408330.821959,54126.918272,1385.643323,89.037108,0.000868
std,60.614796,61.049631,60.149225,60.585451,71.565268,4.787316e+06,0.033596,0.049766,0.505018,1.058612,0.007972,0.008992,0.008396,23.986863,30607.392234,0.008029,2.266187,0.008116,7311.346550,20.458872,82942.255618,3138.471839,289.256821,9.043883,0.018808
min,42.690000,44.190000,41.780000,42.420000,9.590674,8.180000e+05,0.000000,1.000000,0.130000,-1.587300,0.005200,0.000900,-0.007100,-36.980000,137245.000000,0.000400,3.500000,0.017000,127804.000000,210.228000,264465.000000,49408.000000,710.300000,71.870003,-0.136435
25%,78.845000,79.662500,78.155000,78.940000,22.447522,4.037620e+06,0.000000,1.000000,0.320000,0.000000,0.018100,0.003300,0.006400,51.582500,205540.000000,0.000900,4.300000,0.027000,133876.000000,226.665000,346620.000000,51559.000000,1207.675000,80.440002,-0.007938
50%,124.255000,125.020000,123.200000,124.395000,66.639402,6.324432e+06,0.000000,1.000000,0.630000,0.000000,0.023500,0.007300,0.013800,69.910000,221661.000000,0.001600,5.900000,0.029000,141072.000000,237.945000,394200.000000,54021.000000,1313.860000,90.920002,0.001219
75%,196.325000,198.830000,194.027500,196.312500,150.752177,8.829839e+06,0.000000,1.000000,1.200000,0.000000,0.029000,0.014375,0.020000,92.777500,239368.000000,0.009275,8.200000,0.033000,147170.000000,255.211500,451097.000000,56831.000000,1647.412475,96.517498,0.009535
max,278.650000,278.650000,274.440000,278.290000,248.233833,8.438340e+07,0.480000,4.000000,1.980000,25.423700,0.042700,0.047200,0.029100,145.310000,294470.000000,0.038300,14.700000,0.054000,154369.000000,298.012000,632849.000000,60154.000000,2063.187500,114.110001,0.149973
